### Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Tentukan path file di Google Drive
file_path = '/content/drive/MyDrive/Colab Notebooks/Tugas Akhir Fidel RFM/DATASET PT. SSM NEW 2023.csv'

# Membaca dataset
# Specify the encoding as 'latin-1' to handle the problematic byte.
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir Fidel RFM/DATASET PT. SSM NEW 2023.csv', sep=';', encoding='latin-1')
# Menampilkan 5 baris pertama dataset
df.head()

In [ ]:
# Untuk mengubah nama beberapa kolom
df.rename(columns={'ï»¿NAMACUSTOMER': 'NAMACUSTOMER', ' QTY ': 'QTY'}, inplace=True)

In [ ]:
df.info()

###Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir Fidel RFM/DATASET PT. SSM NEW 2023.csv',
                 encoding="utf-8",
                 sep=';', # secara eksplisit tetapkan delimiter
                 on_bad_lines='skip') # lewati baris dengan error

In [ ]:
print(df.info())  # Menampilkan tipe data setiap kolom

In [ ]:
df.rename(columns={' QTY ': 'QTY'}, inplace=True)

### Cek Missing Value

In [ ]:
# Menghitung total missing value di seluruh DataFrame
df.isnull().sum().sum()

In [ ]:
# Menghitung jumlah missing value di setiap kolom
df.isnull().sum()

#### Menangani missing value dengan modus

In [ ]:
import pandas as pd

#Nama Customer
df['NAMACUSTOMER'].fillna('Tidak Diketahui', inplace=True)

#Kota Customer
df['KOTACUSTOMER'].fillna(df['KOTACUSTOMER'].mode()[0], inplace=True)

#No Faktur, TGL, Kode Barang, Nama Barang, Harga Jual
df.dropna(subset=['NOFAKTUR', 'TGL', 'KODEBARANG', 'NAMABARANG', 'HARGAJUAL'], inplace=True)

#QTY, Netto
# Pastikan kolom QTY dan NETTO bertipe numerik sebelum imputasi
df['QTY'] = pd.to_numeric(df['QTY'], errors='coerce')

# Imputasi dengan median
df['QTY'].fillna(df['QTY'].median(), inplace=True)


1. Hapus baris dengan missing value pada NOFAKTUR, TGL, KODEBARANG, 2. NAMABARANG, HARGAJUAL.
3. Imputasi NAMACUSTOMER dengan "Tidak Diketahui".
4. Imputasi KOTACUSTOMER dengan modus.
5. Konversi QTY dan NETTO ke numerik dan imputasi dengan median.

In [ ]:
df.head(100)

#### Mengkonversi TGL ke datetime

In [ ]:
df['TGL'] = pd.to_datetime(df['TGL'], errors='coerce')


In [ ]:
print(df['TGL'].unique())


In [ ]:
df['TGL'] = pd.to_datetime(df['TGL'], format='%d/%m/%Y', errors='coerce')


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['NETTO'] = df['NETTO'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)


In [ ]:
df.info()

In [ ]:
print(df['NETTO'].head())


In [ ]:
# Simpan dataset yang sudah diubah
df.to_csv('dataset_rfm_mentah.csv', index=False, encoding='latin-1')  # Ganti 'dataset_terbaru.csv' dengan nama file yang diinginkan

### Menghtung RFM

In [ ]:
# Definisikan latest_date
latest_date = df['TGL'].max()

# Hitung RFM
rfm = df.groupby('NAMACUSTOMER').agg({
    'TGL': lambda x: (latest_date - x.max()).days,  # Recency (hari sejak transaksi terakhir)
    'NOFAKTUR': 'nunique',  # Frequency (jumlah transaksi unik)
    'NETTO': 'sum'  # Monetary (total belanja)
})

# Ubah nama kolom
rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.reset_index(inplace=True)

In [ ]:
print(rfm.head())

#### Cek Kembali Missing Value setelah perhitungan

In [ ]:
print(rfm.describe())

### Normalisasi Data

#### Normalisasi data dengan StandardScaler


In [ ]:
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])


### Menentukan jumlah cluster optimal

In [ ]:
inertia = []
K = range(1, 11)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Jumlah Cluster')
plt.ylabel('Inertia')
plt.title('Metode Elbow untuk Menentukan K')
plt.show()


In [ ]:
from sklearn.preprocessing import StandardScaler

# Pastikan dataset sudah benar sebelum scaling
print(rfm.describe())  # Sebelum scaling

# Scaling data RFM
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# Cek hasil setelah scaling
import numpy as np
print("Mean setelah scaling:", np.mean(rfm_scaled, axis=0))
print("Std dev setelah scaling:", np.std(rfm_scaled, axis=0))


In [ ]:
from sklearn.cluster import KMeans

# Misal kita pilih K=4
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Lihat jumlah anggota di tiap cluster
print(rfm['Cluster'].value_counts())


In [ ]:
import numpy as np
rfm['Monetary'] = np.log1p(rfm['Monetary'])  # Transformasi untuk Monetary
rfm['Frequency'] = np.log1p(rfm['Frequency'])  # Transformasi untuk Frequency


In [ ]:
from sklearn.preprocessing import StandardScaler

# Pastikan dataset sudah benar sebelum scaling
print(rfm.describe())  # Sebelum scaling

# Scaling data RFM
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# Cek hasil setelah scaling
import numpy as np
print("Mean setelah scaling:", np.mean(rfm_scaled, axis=0))
print("Std dev setelah scaling:", np.std(rfm_scaled, axis=0))

In [ ]:
print(rfm['Cluster'].value_counts())


In [ ]:
from sklearn.metrics import silhouette_score

# Gunakan rfm_scaled dan rfm['Cluster'] sebagai ganti X_scaled dan labels
silhouette_avg = silhouette_score(rfm_scaled, rfm['Cluster'])
print(f"Silhouette Score: {silhouette_avg}")

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
rfm_pca = pca.fit_transform(rfm_scaled)

plt.scatter(rfm_pca[:, 0], rfm_pca[:, 1], c=rfm['Cluster'], cmap='viridis')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Visualisasi Cluster dalam 2D')
plt.colorbar()
plt.show()


### K-Means

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)


In [ ]:
import pandas as pd

# Hitung jumlah pelanggan di setiap cluster
cluster_counts = pd.DataFrame(rfm['Cluster'].value_counts()).reset_index()
cluster_counts.columns = ['Cluster', 'Jumlah Pelanggan']
print(cluster_counts)

In [ ]:
print(rfm['Cluster'].value_counts())


#### Cek distribusi rfm

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Boxplot Recency
plt.figure(figsize=(10,5))
sns.boxplot(x="Cluster", y="Recency", data=rfm)
plt.title("Distribusi Recency per Cluster")
plt.show()

# Boxplot Frequency
plt.figure(figsize=(10,5))
sns.boxplot(x="Cluster", y="Frequency", data=rfm)
plt.title("Distribusi Frequency per Cluster")
plt.show()

# Boxplot Monetary
plt.figure(figsize=(10,5))
sns.boxplot(x="Cluster", y="Monetary", data=rfm)
plt.title("Distribusi Monetary per Cluster")
plt.show()


In [ ]:
rfm['Cluster'].value_counts(normalize=True) * 100


In [ ]:
df.head()

In [ ]:
rfm.set_index('NAMACUSTOMER', inplace=True)


In [ ]:
print(rfm.head())  # Menampilkan 5 baris pertama


In [ ]:
rfm.head()

#### Analisis karakteristik tiap cluster

In [ ]:
rfm.groupby('Cluster').mean()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
sns.boxplot(x="Cluster", y="Monetary", data=rfm)
plt.title("Distribusi Monetary di Setiap Cluster")
plt.show()

- Cluster 0	: 38 hari (recent)	Sering belanja (2.78 kali)	Pengeluaran tinggi (19.36 juta)	Loyal customers → Pelanggan yang aktif dan sering belanja dalam jumlah besar
- Cluster 1 :	266 hari (sangat lama)	Jarang belanja (0.75 kali)	Pengeluaran rendah (14.7 juta)	Lost customers → Pelanggan yang jarang belanja, kemungkinan tidak kembali
- Cluster 2 :	88 hari (menengah)	Kadang belanja (0.93 kali)	Pengeluaran sedang (16.32 juta)	Potential customers → Bisa ditingkatkan loyalitasnya dengan strategi pemasaran


In [ ]:
import numpy as np

# Mengembalikan Frequency ke skala aslinya
rfm["Frequency"] = np.exp(rfm["Frequency"])

# Cek hasil setelah dikembalikan ke skala aslinya
rfm.groupby("Cluster").agg({
    "Recency": "mean",
    "Frequency": "mean",
    "Monetary": "mean"
})


1.  Cluster 0 → Pelanggan Loyal & High-Value
- Sering bertransaksi (21x rata-rata)
- Belanja besar (19.36 juta)
- Perlu dijaga loyalitasnya.

2. Cluster 1 → Pelanggan Pasif (Hampir Hilang)
- Jarang belanja (2x transaksi)
- Sudah lama tidak belanja (266 hari sejak transaksi terakhir)
- Butuh strategi re-engagement (misalnya email promo, cashback, atau penawaran menarik).

3. Cluster 2 → Pelanggan Baru atau Potensial
- Transaksi cukup sering (2-3x)
- Bisa diarahkan menjadi pelanggan loyal dengan program upselling (misalnya promo paket bundling).


In [ ]:
rfm['Cluster'].value_counts()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x=rfm["Cluster"])
plt.title("Distribusi Pelanggan per Cluster")
plt.xlabel("Cluster")
plt.ylabel("Jumlah Pelanggan")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

# Mencoba beberapa jumlah cluster
wcss = []
for i in range(1, 10):  # Coba dari 1 sampai 10 cluster
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(rfm)
    wcss.append(kmeans.inertia_)

# Plot Elbow Method
plt.plot(range(1, 10), wcss, marker='o')
plt.xlabel('Jumlah Cluster')
plt.ylabel('WCSS (Within-Cluster Sum of Squares)')
plt.title('Elbow Method')
plt.show()


In [ ]:
from sklearn.metrics import silhouette_score

# Menghitung Silhouette Score
score = silhouette_score(rfm, rfm['Cluster'])
print(f"Silhouette Score: {score:.4f}")


In [ ]:
import seaborn as sns

plt.figure(figsize=(12,6))
sns.boxplot(x=rfm['Cluster'], y=rfm['Recency'])
plt.title("Distribusi Recency per Cluster")
plt.show()


In [ ]:
import seaborn as sns

plt.figure(figsize=(8,5))
sns.heatmap(rfm.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Korelasi antar Variabel RFM")
plt.show()


In [ ]:
rfm.to_csv('hasil_cluster_rfm.csv', index=True)  # Simpan dengan nama pelanggan sebagai index


In [ ]:
# Pastikan ada kolom tanggal transaksi
rfm['Last_Purchase_Date'] = df.groupby('NAMACUSTOMER')['TGL'].max()


In [ ]:
final_rfm = rfm[['Recency', 'Frequency', 'Monetary', 'Cluster', 'Last_Purchase_Date']]
print(final_rfm.head())


In [ ]:
final_rfm.to_csv('hasil_cluster_rfm.csv', index=True)


#### Analisis produk yang sering dibeli

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("dataset_rfm_mentah.csv")  # Ganti dengan nama file CSV Anda

# Periksa apakah ada kolom 'QTY' atau bukan
if 'QTY ' in df.columns:
    df.rename(columns={'QTY ': 'QTY'}, inplace=True)  # Menghapus spasi ekstra jika ada

# Hitung jumlah pembelian per produk
produk_terlaris = df.groupby("NAMABARANG")["QTY"].sum().reset_index()

# Urutkan dari yang paling banyak dibeli
produk_terlaris = produk_terlaris.sort_values(by="QTY", ascending=False)

# Tampilkan produk terlaris dengan tampilan rapi
print(produk_terlaris.head(10).to_string(index=False))


#### **Interpretasi Data:**
1. Produk-produk dengan nama "CAESAR" dan "YAMATO" mendominasi daftar, yang mengindikasikan bahwa produk-produk ini memiliki permintaan tinggi.

2. Produk kategori kursi dan meja seperti Ayumi Chair No.6 dan Ayumi Desk No.6 juga cukup tinggi permintaannya.


### *Strategi Pemasaran Berdasarkan Data*
Berdasarkan pola pembelian ini, strategi pemasaran yang dapat diterapkan:

#### **Strategi Promosi untuk Produk Terlaris**

- Gunakan strategi retensi pelanggan dengan memberikan diskon atau loyalty program kepada pelanggan yang sering membeli produk ini.

- Terapkan bundle pricing (misalnya, pembelian meja Ayumi mendapatkan diskon untuk kursi Ayumi).

- Iklan digital yang berfokus pada produk-produk best-seller ini melalui Google Ads dan Facebook Ads.

#### **Optimalisasi Stok dan Distribusi**

- Pastikan produk dengan permintaan tinggi selalu tersedia agar tidak kehilangan pelanggan.

- Analisis rantai pasokan untuk mengoptimalkan distribusi produk unggulan ini ke wilayah dengan permintaan tertinggi.

#### **Strategi Upselling dan Cross-Selling**

- Menawarkan produk tambahan atau premium ketika pelanggan membeli produk terlaris.

- Contoh: Jika pelanggan membeli kursi Ayumi Chair No.6, tawarkan meja Ayumi Desk No.6 sebagai paket hemat.

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# Konversi data transaksi menjadi format basket (One-Hot Encoding)
basket = df.groupby(["NOFAKTUR", "NAMABARANG"])["QTY"].sum().unstack().fillna(0)

# Ubah data menjadi binary (1 jika ada pembelian, 0 jika tidak)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# Gunakan Apriori untuk menemukan pola
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)

# Aturan asosiasi berdasarkan confidence
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Tampilkan aturan asosiasi
print(rules[["antecedents", "consequents", "support", "confidence", "lift"]])
